In [2]:
import sys
sys.path.insert(0, r'C:\Users\Public\WPy64-39100\notebooks\Anjana Shaji')
import requests
import pensionpro_api as pp
import pandas as pd
import datetime
from glob import glob
import time
import json
import os
import shutil
import zipfile
from public_vault import Username, Password, OAuth
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from webdriver_manager.chrome import ChromeDriverManager

from webdriver_manager.chrome import ChromeDriverManager

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from pathlib import Path
start = datetime.datetime.now()
print(start)

2024-07-22 14:39:17.492651


In [7]:
current_time = start.time()
current_time
five_pm = datetime.time(16, 0, 0)
if current_time > five_pm:
    print("Current time is greater than 5 PM.")
    raise SystemExit("Script is shutting down")

Current time is greater than 5 PM.


SystemExit: Script is shutting down

C:\Users\Public\WPy64-39100\python-3.9.10.amd64\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
#Plan group id for 'AFS-Partner'
plan_grp_id_list = [464823, 89373, 99916, 438223]
today_date = datetime.date.today()
current_year = today_date.strftime('%Y')
today_date = datetime.datetime.strftime(today_date, "%m-%d-%Y")
email_box = 'automation@nova401k.com'
my_username = Username('DGEM')
my_pw = Password('DGEM')
user = os.getlogin()
download_path = f'C:/Users/{user}/Downloads'
error_path = r'Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\error_report.xlsx'
no_mail_plan_list = []

In [ ]:
def mail_daily_report(file_path, body_text):
    html_head = """
    <html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40">
    <head><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=us-ascii"><meta name=Generator content="Microsoft Word 15 (filtered medium)">
    </head>"""
    
    html_body = f"""
    <body lang=EN-US link="#0563C1" vlink="#954F72" style='word-wrap:break-word'><div class=WordSection1>
    {body_text}<br>
    </div></body></html>
    """.replace('\n','<br>')
    
    toaddr = ['jworms@nova401k.com', 'msvehla@nova401k.com']
    cc_list = []

    html = html_head + html_body

    fromaddr = 'automation@nova401k.com'
    password = 'Rub73595'

    toaddrs = toaddr + cc_list

    if len(toaddrs) == 0:
        x = "No emails to send"
    else:
        msg = MIMEMultipart('alternative')

        msg['From'] = fromaddr
        msg['To'] = ','.join(toaddr)
        msg['CC'] = ','.join(cc_list)
        subject = f'Daily 8955-SSA Daily Status Report'
        msg['Subject'] = subject
        part = MIMEText(html, 'html')
        msg.attach(part)
        
        part1 = MIMEBase('application', "octet-stream")
        part1.set_payload(open(file_path, "rb").read())
        encoders.encode_base64(part1)
        part1.add_header('Content-Disposition', 'attachment; filename= "8955 SSA Daily Status Report.xlsx"')
        msg.attach(part1)


        with smtplib.SMTP('smtp.office365.com', 587) as server:
            server.starttls()
            server.login(fromaddr, password)
            x = server.sendmail(fromaddr, toaddrs, msg.as_string())


In [5]:
def send_email(plan_name, salutation, toaddr, cc_list, am_sig, year):

    html_head = """
    <html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40">
    <head><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=us-ascii"><meta name=Generator content="Microsoft Word 15 (filtered medium)">
    </head>"""
    
    
    html_body = f"""
    <body lang=EN-US link="#0563C1" vlink="#954F72" style='word-wrap:break-word'><div class=WordSection1>
    Dear {salutation},<br>
    We have prepared the {year} Form 8955-SSA for your review.<br>
    This form is used to report plan benefits (or former plan benefits) for two categories of employees to the Social Security Administration: <br>
    * Employees coded as "A" are those who terminated on or before the last date of the prior plan year, but still maintained a plan balance as of the current plan year we are filing. For example, the 12/31/23 8955-SSA would be reporting any 2022 terminated employees with balances as of 12/31/23. <br>
    * Employees coded as "D" are those who were reported on a prior year's Form 8955-SSA and took a full distribution during the current Plan Year.<br>
	<b>ACTION REQUIRED</b><br>
	Please log on to our secure web portal to download a copy of the Form 8955-SSA for your review within 3 business days. This form is located in the ‘My Active Tasks’; once the form has been accessed, it will appear under the ‘Plan Documents’ tab.  If there are no changes, have the plan sponsor and plan administrator sign the Form 8955-SSA and place the signed Form 8955-SSA with the Plan's permanent records; no further action is required on your behalf. If changes are needed, please contact me immediately.<br>
	You may access the form by logging in and selecting the Document Tab at the top:<br>
	<ul type = "1"><li>Log in to PlanSponsorLink through <a href="https://nova401k.plansponsorlink.com/">https://nova401k.plansponsorlink.com/</a> .</li><li>Login Instructions: Your Username is your email address. First time users should click on the 'First time user?' link and enter your email address. PlanSponsorLink will automatically confirm your email address and email you a password.</li></ul>
	Nova 401(k) Associates will electronically file your Form 8955-SSA with the Internal Revenue Service shortly after the 3 business days have lapsed.  Changes requested after the date are subject to additional fees if an amended filing is required.  <br>
    If there are any questions, please do not hesitate to contact me.<br>
    This email was automatically generated.<br>
    {am_sig}</div></body></html>
    """.replace('\n','<br>')

    html = html_head + html_body

    fromaddr = 'clientrelations@nova401k.com'
    password = 'Pas25793'
    # toaddr = ['ashaji@nova401k.com']
    # cc_list = []
    toaddr = [address for address in toaddr if address != None]
    cc_list = [address for address in cc_list if address != None]

    toaddrs = toaddr + cc_list

    if len(toaddrs) == 0:
        x = "No emails to send"
    else:
        msg = MIMEMultipart('alternative')

        msg['From'] = fromaddr
        msg['To'] = ','.join(toaddr)
        msg['CC'] = ','.join(cc_list)
        subject = f'IMPORTANT: {year} Form 8955-SSA Now Available For Your Review - {plan_name}'
        msg['Subject'] = subject
        part = MIMEText(html, 'html')
        msg.attach(part)

        with smtplib.SMTP('smtp.office365.com', 587) as server:
            server.starttls()
            server.login(fromaddr, password)
            x = server.sendmail(fromaddr, toaddrs, msg.as_string())
    return toaddr, cc_list, html_body, subject


In [6]:
def setup_send_mail(plan_id, project_id, signatures, year, afs_flag):
    
    internal_plan_id = pp.get_sysplanid(plan_id)
    plan_name = pp.get_plan_by_planid(internal_plan_id)['Name']
    contacts = pp.get_plan_contact_roles_by_planid(plan_id)
    employees = pp.get_employee_plan_roles_by_planid(plan_id)
    
    primary_contacts = [contact for contact in contacts if contact['RoleType']['DisplayName'] == 'Primary Contact']
    primary_brokers = [contact for contact in contacts if contact['RoleType']['DisplayName'] == 'Primary Broker']
    three_sixteen_admins= [contact for contact in contacts if contact['RoleType']['DisplayName'] == '3(16) Administrator']
    account_managers = [employee for employee in employees if employee['RoleType']['DisplayName'] == 'Administrator']
    
    if primary_brokers:
        primary_broker = primary_brokers[0]
        primary_broker_id = primary_broker.get('Contact', {}).get('Id', None)
        pbroker_email = [broker['Contact']['Email'] for broker in primary_brokers]
    else:
        pbroker_email, primary_broker_id = [[], None]
    if three_sixteen_admins:
        three_sixteen_admin = three_sixteen_admins[0]
        three_sixteen_admin_email = [three_sixteen['Contact']['Email'] for three_sixteen in three_sixteen_admins]
        three_sixteen_admin_id = three_sixteen_admin.get('Contact', {}).get('Id', None)
        three_sixteen_admin_name = three_sixteen_admin.get('Contact', {}).get('FirstName', None)
        three_sixteen_admin_name_last = three_sixteen_admin.get('Contact', {}).get('LastName', None)
        three_six_admin = f'{three_sixteen_admin_name} {three_sixteen_admin_name_last}'
        t_salutation = three_sixteen_admin.get('Contact', {}).get('Salutation', None)
        if t_salutation == None:
            t_salutation = three_sixteen_admin_name
    else:
        three_sixteen_admin_email, three_six_admin, three_sixteen_admin_id = [[], None, None]
    if primary_contacts:
        primary_contact = primary_contacts[0]
        primary_contact_email = [primary['Contact']['Email'] for primary in primary_contacts]
        primary_contact_id = primary_contact.get('Contact', {}).get('Id', None)
        p_salutation = primary_contact.get('Contact', {}).get('Salutation', None)
        if p_salutation == None:
            p_salutation = primary_contact.get('Contact', {}).get('FirstName', None)
    else:
        primary_contact_email, primary_contact_id, p_salutation = [[], None, None]
    if account_managers:
        account_manager = account_managers[0]
        admin_email = [account_manager.get('Contact', {}).get('Email', None)]
        admin_email_sig = admin_email[0]
        admin_name = account_manager.get('Contact', {}).get('FirstName', None)
        admin_name_last = account_manager.get('Contact', {}).get('LastName', None)
        admin_id = account_manager.get('ContactId')
        new_sig = ''
        if [signature['Signature'] for signature in signatures if signature['ContactId'] == admin_id][0] == None:
            admin_sig = admin_name+' '+admin_name_last
            new_sig = admin_sig +'\r\n'+ admin_email_sig
        else:
            admin_sig = [signature['Signature'] for signature in signatures if signature['ContactId'] == admin_id][0].split("\r\n")
        
            for line in admin_sig:
                new_sig = new_sig + line + '\r\n'
    else:
        admin_email = []
        admin_id = None
    new_sig = new_sig.replace("&#10;","\r\n").replace("&#160;"," ").replace("\r\n","<br>").replace("\n","")
    try:
        if afs_flag:
            toaddr = three_sixteen_admin_email
            cc_list = primary_contact_email + pbroker_email
            salutation = t_salutation
            email_output = send_email(plan_name, salutation, toaddr, cc_list, new_sig, year)
            print(f'Email sent for planid {plan_id}!')
        else:
            toaddr = primary_contact_email
            cc_list = pbroker_email
            salutation = p_salutation
            email_output = send_email(plan_name, salutation, toaddr, cc_list, new_sig, year)
            print(f'Email sent for planid {plan_id}!')
    except Exception as e:
        print(e)
        print(f'Email not sent for planid {plan_id}!')
        return True

    to_list = email_output[0]
    cc_list = email_output[1]
    html_body = email_output[2]
    subject = email_output[3]

    note_text = f"""To: {to_list}
        CC: {cc_list}
        Subject: {subject}
        Email: {html_body}
        """
    note_text = note_text[:2000]
    payload = {
    "ProjectID": f"{project_id}", 
    "NoteText": f"{note_text}",
    "ShowOnPSL": False
            }

    x = pp.add_note(payload)
    print(f"Email for {plan_id} has been copied into the project notes!")

In [7]:
def upload_file_to_pro(plan_id, project_id,period_end_date, final_pdf, file_path):
    # Add file to project. ProjectFileTypeId is 583 which is 'Form 8955-SSA'
    print(final_pdf)
    title = final_pdf
    try:
        pp.add_project_file(file_path, project_id, ProjectFileTypeId=583, ShowOnWeb=True, Title= title, Comment='Form 8955-SSA for review', Archived=False, HasBeenWarned=False, EffectiveOn = period_end_date)
        print(f'File added to PRO for plan {plan_id}.')    
    except Exception as e:
        print(e)
        print('No email plan: ',plan_id)
        no_mail_plan_list.append(plan_id)
        files = pp.get_project_files_by_projectid(project_id)
        file = files[0]
        file['ShowOnWeb'] = False
        pp.update_project_file(file)
        title = final_pdf.rsplit('.')[0]
        title = f'{title}2.pdf'
        print('Title 2: ', title)
        pp.add_project_file(file_path, project_id, ProjectFileTypeId=583, ShowOnWeb=True, Title= title, Comment='Form 8955-SSA for review', Archived=False, HasBeenWarned=False, EffectiveOn = period_end_date)
        error_message = str(e)

In [8]:
def get_latest_file(directory):
    ## Reads in the find results. This will let us use the names as selenium lookups

    fr_files = [os.path.join(directory, file) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]

    # Sort the files based on modification time (latest first)
    sorted_fr_files = sorted(fr_files, key=lambda x: os.path.getmtime(x), reverse=True)

    # Get the latest modified file
    latest_find_file = sorted_fr_files[0] if sorted_fr_files else ""

    # Convert the latest_file path to string
    return str(latest_find_file)

In [9]:
df = pp.get_worktray2('Automation', get_all=True)
filt1 = df['task_name'] == 'Delivery of Form 8955-SSA'
filt2 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df = df[filt1 & filt2]
len(df)

342

In [10]:
df

,projid,taskid,task_name,planid,plan_name,proj_name,plan_status,plan_category,plan_type,form5500,plan_end,effective_on,pbgc,proj_due_on
1076,10841868,75558893,Delivery of Form 8955-SSA,4802,GATE Employee Savings Plan,Form 8955-SSA (Automated),Active,PDM - Yes,401(k),Large Plan Filer,12/31,1/1/2016 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
1103,10841904,75559189,Delivery of Form 8955-SSA,10807,Mac Haik Employees' Retirement Plan,Form 8955-SSA (Automated),Active,PDM - Yes,401(k),Large Plan Filer,12/31,1/1/2022 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
1188,10843448,75565543,Delivery of Form 8955-SSA,18355,"Automated Solutions, LLC 401(k) Plan - 002",Form 8955-SSA (Automated),Active,PDM - Yes,Safe Harbor 401(k),5500 SF,12/31,1/1/2023 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
1201,10843519,75566072,Delivery of Form 8955-SSA,18848,Department of Commerce Federal Credit Union 40...,Form 8955-SSA (Automated),Active,PDM - Yes,Safe Harbor 401(k),5500 SF,12/31,1/1/2023 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
1219,10849699,75598291,Delivery of Form 8955-SSA,16275,"CNHI, LLC 401(k) Plan",Form 8955-SSA (Automated),Active,PDM - Yes,401(k),Large Plan Filer,12/31,1/1/2021 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,10900605,75851601,Delivery of Form 8955-SSA,18530,"Banda Group International, LLC 401(k) Profit S...",Form 8955-SSA (Automated),Active,PDM - Yes,Safe Harbor 401(k),5500 SF,12/31,1/1/2022 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
2027,10900606,75851609,Delivery of Form 8955-SSA,16387,"Alamar Healthcare, Inc. 401(k) Profit Sharing ...",Form 8955-SSA (Automated),Active,PDM - Yes,Safe Harbor 401(k),5500 SF,12/31,1/1/2020 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
2030,10900612,75851642,Delivery of Form 8955-SSA,5316,Mathers Construction Co. 401(k) Plan,Form 8955-SSA (Automated),Active,PDM - Yes,Safe Harbor 401(k),5500 SF,12/31,1/1/2018 12:00:00 AM,N/A,10/15/2024 12:00:00 AM
2033,10900616,75851675,Delivery of Form 8955-SSA,7474,Pulmonary Critical Care & Sleep Medicine Consu...,Form 8955-SSA (Automated),Active,PDM - Yes,Safe Harbor 401(k),5500 SF,12/31,1/1/2007 12:00:00 AM,N/A,10/15/2024 12:00:00 AM


In [11]:
now = datetime.datetime.now()
previous_year = str(int(now.strftime("%Y"))-1)
o365_directory = Path(r'C:\Users\Public\WPy64-39100\notebooks\scheduler')
generated_import_file = 'AA'
# logs in using the Automation login
if generated_import_file != "No import":

    browser = webdriver.Chrome(ChromeDriverManager().install())
    # Login-----------------------------------------------------------------------
    browser.get('https://dgem.asc-net.com/ascidoc/login.aspx')

    tpakey = browser.find_element_by_name('tbTPA')
    tpakey.send_keys('asc438')

    username = browser.find_element_by_name('tbUsername')
    username.send_keys(my_username)               

    WebDriverWait(browser,10).until(ec.visibility_of_element_located((By.NAME,'tbPassword')))
    password = browser.find_element_by_name('tbPassword')
    password.send_keys(my_pw)

    loginbutton = browser.find_element_by_name('btnLogin')
    loginbutton.click()
    
    
    try:
        # Click to send verification code via "Email"
        WebDriverWait(browser,10).until(ec.visibility_of_element_located((By.XPATH,'//*[@id="MFAUC_rbEmail"]')))
        browser.find_element_by_xpath('//*[@id="MFAUC_rbEmail"]').click()

        # Click SEND
        browser.find_element_by_xpath('//*[@id="MFAUC_btnSend"]').click()
        
        if generated_import_file != "No import":
            from O365 import Account, FileSystemTokenBackend

            credentials = OAuth('o365_client_id'), OAuth('o365_secret_value')

            token_backend = FileSystemTokenBackend(token_path=o365_directory,
                                                   token_filename='oauth_token.txt')
            account = Account(credentials,
                              token_backend=token_backend,
                              scopes = ['basic','message_all','mailbox'])

            if not account.is_authenticated:
                account.authenticate()

            time.sleep(60)

            mailbox = account.mailbox(resource='automation@nova401k.com')

            inbox = mailbox.inbox_folder().get_messages()

            for message in inbox:
                msg_sender = str(message.sender)
                if (msg_sender == 'ASC (no-reply@pension-plan-emails.com)'
                    and (msg_subject := message.subject) == 'Verification Code'):
                    msg_body = message.body
                    regex = re.search(r'(?<=>)(?P<code>\d{6})(?=<)',msg_body) # find 6-digit num if between < and >
                    verification_code = regex.group('code')
                    message.mark_as_read()
                    break

            # Enter Verification code
            browser.find_element_by_xpath('//*[@id="MFAUC_tbCode"]').send_keys(verification_code)

            # Click verify
            browser.find_element_by_xpath('//*[@id="MFAUC_btnVerify"]').click()

            # Click Yes
            WebDriverWait(browser,10).until(ec.visibility_of_element_located((By.XPATH,'//*[@id="MFAUC_rbYes"]')))
            browser.find_element_by_xpath('//*[@id="MFAUC_rbYes"]').click()

            # Click "Continue"
            browser.find_element_by_xpath('//*[@id="MFAUC_btnContinue"]').click()

        else:
            exit()
    except:
        # no MFA required
        pass

In [12]:
# Download Find results
time.sleep(10)
errorlist = []

browser.get('https://dgem.asc-net.com/ascidoc/Find.aspx')

time.sleep(10)
form_type = browser.find_element(by='id',value='ddDocumentType').send_keys("8955-SSA")
status = browser.find_element(by='id',value='dd_SSA_status').send_keys("All")
# checkbox = browser.find_element(by='name',value='cb_SSA_statusChangedAfter').click()

time.sleep(5)
checkbox = browser.find_element(by='id',value='lbtnNext_5500').click()

time.sleep(20)

dropdown_menu = browser.find_element(by='name',value='lbActionSSA').send_keys("Export Find Results")

find_next = browser.find_element(by='id',value='lbtnClientNextSSA').click()

time.sleep(10)

link_check = False

while link_check is False:
    if "Export is complete." in browser.page_source:
        time.sleep(1)
        browser.find_element(by='link text',value='here').click()
        link_check = True
        
time.sleep(20)


In [13]:
## Reads in the find results. This will let us use the names as selenium lookups
latest_find_result = get_latest_file(download_path)
results = pd.read_html(latest_find_result, converters={"EIN":str,"Plan #":str})
df_results = results[0]
df_results['Identifier'] = df_results['Identifier'].fillna(0)
df_results['Identifier'] = df_results['Identifier'].astype(int)
df_results['Lookup'] = df_results['Plan #'] + df_results['EIN']
os.remove(latest_find_result)

df_results

,Name,EIN,Year,Plan #,End Date,Status,Date Modified,5500 extension Date,Identifier,Lookup
0,1800 RADIATOR 401(K) PLAN,061780522,2023,002,12/31/2023,Valid,7/17/2024,NaN,12077,002061780522
1,2-C EQUIPMENT 401(K) PLAN,200414493,2023,001,12/31/2023,Valid,7/19/2024,NaN,4165,001200414493
2,"2014 FAIRVIEW AVE., LLC 401(K) PROFIT SHARING ...",260313093,2023,001,12/31/2023,Draft,7/17/2024,NaN,18201,001260313093
3,"2020 EXHIBITS, INC. 401(K) PROFIT SHARING PLAN",760664110,2023,001,12/31/2023,Valid,7/17/2024,NaN,5642,001760664110
4,"21ST CENTURY HEALTHCARE, INC. 401(K) PLAN",860733416,2023,001,12/31/2023,Valid,7/16/2024,NaN,15355,001860733416
...,...,...,...,...,...,...,...,...,...,...
2644,ZINSMEYER STRUCTURAL STEEL 401(K) PLAN,822782974,2023,001,12/31/2023,Valid,7/16/2024,NaN,4702,001822782974
2645,ZION INNOVATION CORP 401(K) PLAN,464643637,2023,001,12/31/2023,Good,7/19/2024,NaN,11184,001464643637
2646,"ZIVETZ, SCHWARTZ & SALTSMAN CPAS, APC 401(K) P...",952808993,2023,001,12/31/2023,Valid,7/19/2024,NaN,6090,001952808993
2647,Zonder Family Law 401(k) Profit Sharing Plan,824034066,2023,001,12/31/2023,Valid,7/16/2024,NaN,16476,001824034066


In [14]:
valid_plan_list = []
planlist = list(set(df_results['Identifier'].tolist()))
for index, row in df.iterrows():
    plan_id = int(row['planid'])
    try:
        plan_name = df_results.loc[df_results['Identifier'] == plan_id, 'Name'].iloc[0]
    except Exception as e:
        print(e)
    valid_plan_list.append(plan_name)
print(len(valid_plan_list))
valid_plan_list

342


['GATE EMPLOYEE SAVINGS PLAN',
 "MAC HAIK EMPLOYEES' RETIREMENT PLAN",
 'AUTOMATED SOLUTIONS, LLC 401(K) PLAN - 001',
 'DEPARTMENT OF COMMERCE FEDERAL CREDIT UNION 401(K) PLAN',
 'CNHI, LLC 401(K) PLAN',
 'PARTNERS INFORMATION TECHNOLOGY 401(K) PROFIT SHARING PLAN',
 'GREATER DALLAS & ASSOCIATES 401(K) PLAN',
 'MPE 401(k) Plan',
 'MOUNTAIN VALLEY MECHANICAL 401(K) PLAN',
 'COBRATEC KNIVES 401(K) PLAN',
 'GRAHAM HART HOME BUILDER 401(K) PLAN',
 'DOFST, LLC RETIREMENT SAVINGS PLAN',
 'SHOREVIEW INDUSTRIES IV LLC 401(K) PLAN',
 'MCPOWELL MANAGEMENT, INC. 401(K) PLAN',
 'EXODYNE TECHNOLOGIES, INC. 401(K) PLAN',
 'RIVER PLATE CORPORATION 401(K) PROFIT SHARING PLAN',
 'GWENDOLYN D. KINES, DDS, LLC 401(K) PLAN',
 'INSIGHT FINANCIAL STAFFING & SEARCH GROUP LLC 401(K) PLAN',
 'CNPC USA CORP. RETIREMENT PLAN',
 'MURRAY & MURRAY 401(K) PLAN',
 'MINDSET GROUP 401(K) PLAN',
 'H&H INC 401(K) PROFIT SHARING PLAN',
 'JOTRON USA, INC. 401(K) PLAN',
 'CAS CONSULTING 401(k) PLAN',
 'NOVUS ARCHITECTS INC.

In [19]:
# Download the batch file for 8955
# Request PDFs to be emailed to automation@nova401k.com

browser.get('https://dgem.asc-net.com/ascidoc/Find.aspx')

form_type = browser.find_element(by='id',value='ddDocumentType').send_keys("8955-SSA")
time.sleep(2)
status = browser.find_element(by='id',value='dd_SSA_status').send_keys("All")
# checkbox = browser.find_element(by='name',value='cb_SSA_statusChangedAfter').click()

time.sleep(5)
checkbox = browser.find_element(by='id',value='lbtnNext_5500').click()

In [20]:
time.sleep(10)
b=0
errorlist = []
for plan in valid_plan_list[b:]:
    print(plan)
#     plan = plan.upper()
    try:
        browser.find_element(by='xpath',value=f'//td[contains(text(),"{plan}")]/ancestor::tr[1]//input[@type = "checkbox"]').click()
        b+=1
        print(plan, b, valid_plan_list.index(plan))

    except:
        errorlist.append(plan)
        continue

dropdown_menu = browser.find_element(by='name',value='lbActionSSA').send_keys("View PDF (5500VS Batch)")
find_next = browser.find_element(by='id',value='lbtnClientNextSSA').click()

time.sleep(10)

email_box = browser.find_element(by='id',value='tbEmail').send_keys("automation@nova401k.com")
email_next = browser.find_element(by='id',value='btnEmailInputNext').click()
email_requested_time = datetime.now()

GATE EMPLOYEE SAVINGS PLAN
GATE EMPLOYEE SAVINGS PLAN 1 0
MAC HAIK EMPLOYEES' RETIREMENT PLAN
MAC HAIK EMPLOYEES' RETIREMENT PLAN 2 1
AUTOMATED SOLUTIONS, LLC 401(K) PLAN - 001
AUTOMATED SOLUTIONS, LLC 401(K) PLAN - 001 3 2
DEPARTMENT OF COMMERCE FEDERAL CREDIT UNION 401(K) PLAN
DEPARTMENT OF COMMERCE FEDERAL CREDIT UNION 401(K) PLAN 4 3
CNHI, LLC 401(K) PLAN
CNHI, LLC 401(K) PLAN 5 4
PARTNERS INFORMATION TECHNOLOGY 401(K) PROFIT SHARING PLAN
PARTNERS INFORMATION TECHNOLOGY 401(K) PROFIT SHARING PLAN 6 5
GREATER DALLAS & ASSOCIATES 401(K) PLAN
GREATER DALLAS & ASSOCIATES 401(K) PLAN 7 6
MPE 401(k) Plan
MPE 401(k) Plan 8 7
MOUNTAIN VALLEY MECHANICAL 401(K) PLAN
MOUNTAIN VALLEY MECHANICAL 401(K) PLAN 9 8
COBRATEC KNIVES 401(K) PLAN
COBRATEC KNIVES 401(K) PLAN 10 9
GRAHAM HART HOME BUILDER 401(K) PLAN
GRAHAM HART HOME BUILDER 401(K) PLAN 11 10
DOFST, LLC RETIREMENT SAVINGS PLAN
DOFST, LLC RETIREMENT SAVINGS PLAN 12 11
SHOREVIEW INDUSTRIES IV LLC 401(K) PLAN
SHOREVIEW INDUSTRIES IV LLC 401

PF WATERWORKS L.P. RETIREMENT PLAN 98 97
PHOENIX INDUSTRIAL SERVICES, INC. 401(K) PLAN
PHOENIX INDUSTRIAL SERVICES, INC. 401(K) PLAN 99 98
PRO BOX 401(K) PLAN & TRUST
PRO BOX 401(K) PLAN & TRUST 100 99
ROGGE DUNN 401(K) PLAN
ROGGE DUNN 401(K) PLAN 101 100
S&M ZIMMERMAN ENTERPRISES 401(K) PLAN
S&M ZIMMERMAN ENTERPRISES 401(K) PLAN 102 101
SHERIFFS' ASSOCIATION OF TEXAS, INC 401(K)/PROFIT SHARING PLAN
SHERIFFS' ASSOCIATION OF TEXAS, INC 401(K)/PROFIT SHARING PLAN 103 102
SOUTHERN INNOVATIONS, LLC 401(K) PROFIT SHARING PLAN
SOUTHERN INNOVATIONS, LLC 401(K) PROFIT SHARING PLAN 104 103
STILES ELECTRIC, INC. 401(K) PLAN
STILES ELECTRIC, INC. 401(K) PLAN 105 104
STRUKMYER 401(K) PLAN
STRUKMYER 401(K) PLAN 106 105
THRIVE 401(K) & PROFIT SHARING PLAN
THRIVE 401(K) & PROFIT SHARING PLAN 107 106
TNA NA MANUFACTURING, INC. 401(K) AND PROFIT SHARING PLAN
TNA NA MANUFACTURING, INC. 401(K) AND PROFIT SHARING PLAN 108 107
TOPA TOPA BREWING CO. 401(K) PLAN
TOPA TOPA BREWING CO. 401(K) PLAN 109 108
ULTI

VORTEX CONSTRUCTION LLC 401(K) PROFIT SHARING PLAN AND TRUST 196 195
WORLDWIDE POWER PRODUCTS 401(K) PLAN
WORLDWIDE POWER PRODUCTS 401(K) PLAN 197 196
ASPEN HELICOPTERS, INC. 401(K) RETIREMENT SAVINGS PLAN
ASPEN HELICOPTERS, INC. 401(K) RETIREMENT SAVINGS PLAN 198 197
CM MEDICAL 401(K) PLAN
CM MEDICAL 401(K) PLAN 199 198
COMPOSITE RESOURCES, INC. 401(K) PLAN
COMPOSITE RESOURCES, INC. 401(K) PLAN 200 199
EPSTEIN CONSTRUCTION 401(K) PLAN
EPSTEIN CONSTRUCTION 401(K) PLAN 201 200
FLESHERS FACILITIES 401(K) RETIREMENT PLAN
FLESHERS FACILITIES 401(K) RETIREMENT PLAN 202 201
HICKORY PARK FURNITURE GALLERIES, INC. RETIREMENT SAVINGS PLAN
HICKORY PARK FURNITURE GALLERIES, INC. RETIREMENT SAVINGS PLAN 203 202
LAW OFFICE OF BRANDON WHITE 401(K) PLAN
LAW OFFICE OF BRANDON WHITE 401(K) PLAN 204 203
LEWICKY, O'CONNOR, HUNT & MEISER 401(k) PLAN
LEWICKY, O'CONNOR, HUNT & MEISER 401(k) PLAN 205 204
NEXTMUNE 401(K) PLAN
NEXTMUNE 401(K) PLAN 206 205
SANDIA NEUROLOGY GROUP 401(K) PLAN
SANDIA NEUROLOGY GRO

MMP LLC 401(K) PROFIT SHARING PLAN TRUST 294 293
J&L OILFIELD SERVICES SAVINGS INCENTIVE PLAN
J&L OILFIELD SERVICES SAVINGS INCENTIVE PLAN 295 294
VITAL HOME SECURITY AND AUTOMATION, INC 401(K) PROFIT SHARING PLAN
VITAL HOME SECURITY AND AUTOMATION, INC 401(K) PROFIT SHARING PLAN 296 295
ALENCAR FAMILY DENTISTRY RETIREMENT PLAN
ALENCAR FAMILY DENTISTRY RETIREMENT PLAN 297 296
BOYDS PAINTING SERVICE, INC. 401(K) PLAN
BOYDS PAINTING SERVICE, INC. 401(K) PLAN 298 297
CANYON PACIFIC INSURANCE SERVICES 401(K) PLAN
CANYON PACIFIC INSURANCE SERVICES 401(K) PLAN 299 298
DMRE 401(K) PLAN
DMRE 401(K) PLAN 300 299
E-DIALYSIS 401(K) PLAN
E-DIALYSIS 401(K) PLAN 301 300
ENERGY STEEL PRODUCTS, LLC RETIREMENT AND 401(K) PLAN
ENERGY STEEL PRODUCTS, LLC RETIREMENT AND 401(K) PLAN 302 301
NS804 401(K) PLAN
NS804 401(K) PLAN 303 302
PGH ENGINEERS 401(K) PLAN
PGH ENGINEERS 401(K) PLAN 304 303
SAN MARTINO IMPORTS INC. 401(K) PROFIT SHARING PLAN
SAN MARTINO IMPORTS INC. 401(K) PROFIT SHARING PLAN 305 304
CRA

In [21]:
# part to get PDF zip file from the email.

time.sleep(60)
quit_flag = False

email_found=False
timeout = time.time() + 60*20   # timeout 15 minutes from now
from O365 import Account, FileSystemTokenBackend

credentials = OAuth('o365_client_id'), OAuth('o365_secret_value')

token_backend = FileSystemTokenBackend(token_path=o365_directory,
                                       token_filename='oauth_token.txt')
account = Account(credentials,
                  token_backend=token_backend,
                  scopes = ['basic','message_all','mailbox'])

if not account.is_authenticated:
    account.authenticate()
    
mailbox = account.mailbox(resource=email_box)
while email_found is False:
    time.sleep(10)
    inbox = mailbox.inbox_folder().get_messages()
    if time.time() > timeout:
        print("timeout")
        quit_flag = True
        browser.quit()
#         break
    for message in inbox:
        msg_sender = str(message.sender)
        msg_time_sent = message.sent #get time message was sent
        msg_time_sent = msg_time_sent.replace(tzinfo=None)
        recent_email = msg_time_sent > email_requested_time
        if (msg_sender == '8955-SSA PDFs (support@pension-plan-emails.com)'
            and (msg_subject := message.subject) == 'Completed Batch PDF creation for 8955-SSA forms'
           and recent_email is True):
            msg_body = message.body
            regex = re.search(r'<a class="fill-div" href="([^"]+)"',msg_body) # find sendgrid url
            pdf_download = regex.group(1)
            message.mark_as_read()
            email_found=True
            break
            
if email_found is True:
    if quit_flag:
        browser = webdriver.Chrome(ChromeDriverManager().install())
        # Login-----------------------------------------------------------------------
        browser.get('https://dgem.asc-net.com/ascidoc/login.aspx')

        tpakey = browser.find_element_by_name('tbTPA')
        tpakey.send_keys('asc438')

        username = browser.find_element_by_name('tbUsername')
        username.send_keys(my_username)               

        WebDriverWait(browser,10).until(ec.visibility_of_element_located((By.NAME,'tbPassword')))
        password = browser.find_element_by_name('tbPassword')
        password.send_keys(my_pw)

        loginbutton = browser.find_element_by_name('btnLogin')
        loginbutton.click()
    browser.get(pdf_download)
    print('Batch downloaded')

else:
    raise Exception("No email found")


time.sleep(10)

timeout


Exception: No email found

In [16]:
## Extract the zip files

# Get latest file in dl directory
latest_file_string = get_latest_file(download_path)
print(latest_file_string)

try:
    dirname = latest_file_string.split(".zip")[0]
    print(dirname)
    
except:
    raise Exception("No zip file found")
    
with zipfile.ZipFile(latest_file_string, 'r') as zip_ref:
    zip_ref.extractall(dirname)
time.sleep(20)
os.remove(latest_file_string)
unzipped_pdfs = os.listdir(dirname)
for file in unzipped_pdfs:
    shutil.copy(f"{dirname}/{file}", f'Y:/ASC/Exported Reports/8955 Automation/PDF Downloads/{file}')
    os.remove(f"{dirname}/{file}")
                
os.removedirs(dirname)

C:/Users/ashaji/Downloads\batch8955SSA_8dca755ff819a59.zip
C:/Users/ashaji/Downloads\batch8955SSA_8dca755ff819a59


In [17]:
browser.quit()

In [18]:
# Rename the 8955 forms

pdf_root = f'Y:/8955/{current_year}/Automation/SSA Production/PDF Downloads'
os.chdir('Y:/ASC/Exported Reports/8955 Automation/PDF Downloads')
pdf_folder = os.listdir()
pdf_folder = [file for file in pdf_folder if file.endswith(".pdf")]

pdf_folder = ([file for file in pdf_folder if 'FormSSA_' in file and not '__' in file and not 'Identifier' in file])
if len(pdf_folder) > 0:        
    for file in pdf_folder:
        tpa_planid = file.split("_")[1]
        if "8955SSA.pdf" not in file: # update filenames here
            try:
                os.rename(file, f"{tpa_planid}_8955SSA.pdf")
    
            except Exception as e:
                print(e)
    
    
print('Done!')

# get folder contents again after renaming
pdf_folder = os.listdir()
pdf_folder = [file for file in pdf_folder if file.endswith(".pdf")]
target_files = ([file for file in pdf_folder if not '__' in file and not 'Identifier' in file])
final_pdfs = target_files[:]
len(final_pdfs)

Done!


412

In [19]:
#create a current year folder to save the downloaded PDFs.
parent_dir = 'Y:/8955'
directory = f'{current_year}/Automation/SSA Production/PDF Downloads/Dated Form Downloads/{today_date} PDF Downloads'
save_pdf_path = os.path.join(parent_dir, directory)
try: 
    os.makedirs(save_pdf_path, exist_ok = True) 
    print("Directory '%s' created successfully" % directory) 
except OSError as error: 
    print("Directory '%s' can not be created" % directory) 
save_pdf_path

Directory '2024/Automation/SSA Production/PDF Downloads/Dated Form Downloads/07-18-2024 PDF Downloads' created successfully


'Y:/8955\\2024/Automation/SSA Production/PDF Downloads/Dated Form Downloads/07-18-2024 PDF Downloads'

In [20]:
print(len(final_pdfs))

412


In [21]:
source = r'Y:\ASC\Exported Reports\8955 Automation\PDF Downloads'
os.chdir('Y:/ASC/Exported Reports/8955 Automation/PDF Downloads')
df_error_log = pd.read_excel(error_path)
for file in final_pdfs:
    plan_id = file.split('_')[0]
    try:
        move_directory = r'G:'
        client_folder = ''
        pdf_folder = os.listdir()
        file_string = f'{plan_id}_'
        pdf_folder = [file for file in pdf_folder if file.endswith(".pdf")]
        target_files = ([file for file in pdf_folder if not '__' in file and not 'Identifier' in file and file_string in file])
        final_pdf = target_files[0]
        print(final_pdf)
        projects = pp.get_projects_by_planid(plan_id, filters="Name eq 'Form 8955-SSA (Automated)'")
        project = [project for project in projects if project['CompletedOn'] is None]
        if len(project) > 0:
            project = project[0]
        period_end_date = project['PeriodEnd']
        project_id = project['Id']
        project_start_date = project['PeriodStart']
        project_start_year = project_start_date.rsplit(' ')[0].rsplit('/')[2]
        file_path = f'{source}\{final_pdf}'
        upload_file_to_pro(plan_id, project_id, period_end_date, final_pdf, file_path)
        for folder in os.listdir(move_directory):
            if folder.split()[0] == plan_id:
                client_folder = folder
        source_path = os.path.join(source, final_pdf)
        destination_folder = f'{move_directory}\{client_folder}\\{project_start_year}\\5500\\8955'
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)
        print(destination_folder)
        destination_path = os.path.join(destination_folder, final_pdf)
        shutil.copyfile(source_path, destination_path)
        shutil.copyfile(source_path, f'{save_pdf_path}\{final_pdf}')
        time.sleep(10)
        os.remove(final_pdf)
        df_error_log.loc[df_error_log['TPA Plan ID'].astype(str).str.contains(str(plan_id)), 'Form Downloaded'] = 'Yes'
    except Exception as e:
        print(plan_id, 'error', e)
print('Done!')

10049_8955SSA.pdf
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
10049_8955SSA.pdf
here
File added to PRO for plan 10049.
G:\10049 MSE Landscape Professionals, Inc. 401(k) Plan\2023\5500\8955
10061_8955SSA.pdf
10061_8955SSA.pdf
here
File added to PRO for plan 10061.
G:\10061 March Products Inc (California Umbrella Inc.) 401(k) Plan\2023\5500\8955
10085_8955SSA.pdf
10085_8955SSA.pdf
here
File added to PRO for plan 10085.
G:\10085 All Industries 401(k) Plan\2023\5500\8955
10097_8955SSA.pdf
10097_8955SSA.pdf
here
File added to PRO for plan 10097.
G:\10097 Alava Consulting 401(k) Plan\2023\5500\8955
10098_8955SSA.pdf
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
10

File added to PRO for plan 10949.
G:\10949 Northern Arizona Wind & Sun 401(k) Plan & Trust\2023\5500\8955
10963_8955SSA.pdf
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
10963_8955SSA.pdf
here
File added to PRO for plan 10963.
G:\10963 Innovative Construction Concepts, Inc. 401(k) Profit Sharing Plan\2023\5500\8955
11041_8955SSA.pdf
11041_8955SSA.pdf
here
File added to PRO for plan 11041.
G:\11041 The Fricks Company 401(k) Plan\2023\5500\8955
11044_8955SSA.pdf
11044_8955SSA.pdf
here
File added to PRO for plan 11044.
G:\11044 Harry's on the Hill, LLC 401(k) Plan\2023\5500\8955
11047_8955SSA.pdf
11047_8955SSA.pdf
here
File added to PRO for plan 11047.
G:\11047 Darren B. Crosbie, D.D.S., P.A. 401(k) Plan\2023\5500\8955
11086_8955SSA.pdf
11086_8955SSA.pdf
here
File added to PRO for plan 11086.
G:\11086 Old Pueblo Community Ser

12006_8955SSA.pdf
12006_8955SSA.pdf
here
File added to PRO for plan 12006.
G:\12006 The Company Retirement Plan\2023\5500\8955
12012_8955SSA.pdf
12012_8955SSA.pdf
here
Project File save Rule Violations | Rule: A file with this title was already found on this project, please update title
No email plan:  12012
Title 2:  12012_8955SSA2.pdf
here
G:\12012 Premier Oncology Consultants, P.A. 401(k) Plan\2023\5500\8955
12014_8955SSA.pdf
12014_8955SSA.pdf
here
File added to PRO for plan 12014.
G:\12014 Primrose Oil Company Employee Savings Plan\2023\5500\8955
12023_8955SSA.pdf
12023_8955SSA.pdf
here
File added to PRO for plan 12023.
G:\12023 Schwob Companies 401(k) Plan\2023\5500\8955
12024_8955SSA.pdf
12024_8955SSA.pdf
here
File added to PRO for plan 12024.
G:\12024 Cook Induction Heating Co., Inc. Profit Sharing Plan\2023\5500\8955
12037_8955SSA.pdf
12037_8955SSA.pdf
here
File added to PRO for plan 12037.
G:\12037 T.G. Miller, P.C. Retirement Plan\2023\5500\8955
12047_8955SSA.pdf
12047_8955SS

15745_8955SSA.pdf
15745_8955SSA.pdf
here
File added to PRO for plan 15745.
G:\15745 Fleschler & Phillips Medical Group 401(k) Plan\2023\5500\8955
15811_8955SSA.pdf
15811_8955SSA.pdf
here
File added to PRO for plan 15811.
G:\15811 Texas Premier Drywall 401(k) Plan\2023\5500\8955
15912_8955SSA.pdf
15912_8955SSA.pdf
here
File added to PRO for plan 15912.
G:\15912 Power Engineering Co., Inc. 401(k) Profit Sharing Plan\2023\5500\8955
15917_8955SSA.pdf
15917_8955SSA.pdf
here
File added to PRO for plan 15917.
G:\15917 Analytical Industries, Inc. 401(k) Profit Sharing Plan\2023\5500\8955
15930_8955SSA.pdf
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
15930_8955SSA.pdf
here
File added to PRO for plan 15930.
G:\15930 Brawley & Associates 401(k) Plan\2023\5500\8955
15971_8955SSA.pdf
15971_8955SSA.pdf
here
File added to PRO for plan 1

18076_8955SSA.pdf
18076_8955SSA.pdf
here
File added to PRO for plan 18076.
G:\18076 Red E Services 401(k) Plan\2023\5500\8955
18131_8955SSA.pdf
18131_8955SSA.pdf
here
File added to PRO for plan 18131.
G:\18131 One Stop Shop 401(k) Plan\2023\5500\8955
18173_8955SSA.pdf
18173_8955SSA.pdf
here
File added to PRO for plan 18173.
G:\18173 Nothing Bundt Cakes 401(k) Plan\2023\5500\8955
18263_8955SSA.pdf
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
18263_8955SSA.pdf
here
File added to PRO for plan 18263.
G:\18263 Hill & Wood Funeral Service, Inc. 401(k) Profit Sharing Plan\2023\5500\8955
18360_8955SSA.pdf
18360_8955SSA.pdf
here
File added to PRO for plan 18360.
G:\18360 Automated Solutions LLC 401(k) Plan -001\2023\5500\8955
18424_8955SSA.pdf
18424_8955SSA.pdf
here
File added to PRO for plan 18424.
G:\18424 J. Anthony Custom Homes 401(k) Plan\2023\5500\8955
18506_8955SSA.pdf
18506_8955SSA.pdf
here
File added to PRO for plan 18506.
G:

G:\2726 Cody Pools 401(k) Plan\2023\5500\8955
2748_8955SSA.pdf
2748_8955SSA.pdf
here
File added to PRO for plan 2748.
G:\2748 Brennan Financial Services 401(k) Plan\2023\5500\8955
2758_8955SSA.pdf
2758_8955SSA.pdf
here
File added to PRO for plan 2758.
G:\2758 Northwest Volunteer Fire Department\2023\5500\8955
2793_8955SSA.pdf
2793_8955SSA.pdf
here
File added to PRO for plan 2793.
G:\2793 M-Spex 401(k) Plan\2023\5500\8955
2818_8955SSA.pdf
2818_8955SSA.pdf
here
File added to PRO for plan 2818.
G:\2818 Tejas Sales 401(k) Plan\2023\5500\8955
2868_8955SSA.pdf
2868_8955SSA.pdf
here
File added to PRO for plan 2868.
G:\2868 Buckingham Center 401(k)\2023\5500\8955
2885_8955SSA.pdf
2885_8955SSA.pdf
here
File added to PRO for plan 2885.
G:\2885 Agricultural Services, Inc. PSP\2023\5500\8955
2906_8955SSA.pdf
2906_8955SSA.pdf
here
File added to PRO for plan 2906.
G:\2906 S & S Promotions\2023\5500\8955
2922_8955SSA.pdf
2922_8955SSA.pdf
here
File added to PRO for plan 2922.
G:\2922 Shield Air Soluti

4379_8955SSA.pdf
4379_8955SSA.pdf
here
File added to PRO for plan 4379.
G:\4379 A's Plumbing Service 401(k) Plan\2023\5500\8955
4395_8955SSA.pdf
4395_8955SSA.pdf
here
File added to PRO for plan 4395.
G:\4395 Patel & Dalrymple\2023\5500\8955
4399_8955SSA.pdf
4399_8955SSA.pdf
here
File added to PRO for plan 4399.
G:\4399 New South Properties 401(k) Profit Sharing Plan\2023\5500\8955
4434_8955SSA.pdf
4434_8955SSA.pdf
here
File added to PRO for plan 4434.
G:\4434 US Resources, Inc Profit Sharing Plan\2023\5500\8955
4452_8955SSA.pdf
4452_8955SSA.pdf
here
File added to PRO for plan 4452.
G:\4452 Rankin USA, Inc. 401(k) Plan\2023\5500\8955
4552_8955SSA.pdf
4552_8955SSA.pdf
here
File added to PRO for plan 4552.
G:\4552 Little Buddy 401(k) Plan\2023\5500\8955
4594_8955SSA.pdf
4594_8955SSA.pdf
here
File added to PRO for plan 4594.
G:\4594 Fernelius Chevrolet\2023\5500\8955
4611_8955SSA.pdf
4611_8955SSA.pdf
here
File added to PRO for plan 4611.
G:\4611 Patin Construction 401(k) Plan\2023\5500\895

File added to PRO for plan 6086.
G:\6086 DMP Inc. 401(k) Plan\2023\5500\8955
6105_8955SSA.pdf
6105_8955SSA.pdf
here
File added to PRO for plan 6105.
G:\6105 Stewart Bradbury 401(k) Plan\2023\5500\8955
6119_8955SSA.pdf
6119_8955SSA.pdf
here
File added to PRO for plan 6119.
G:\6119 Pyramids Pharmacy 401(k) Plan\2023\5500\8955
6147_8955SSA.pdf
6147_8955SSA.pdf
here
File added to PRO for plan 6147.
G:\6147 LADD Equipment LP 401(k) Profit Sharing Plan\2023\5500\8955
6154_8955SSA.pdf
6154_8955SSA.pdf
here
File added to PRO for plan 6154.
G:\6154 Blank Canvas 401(k) Profit Sharing Plan\2023\5500\8955
6171_8955SSA.pdf
6171_8955SSA.pdf
here
File added to PRO for plan 6171.
G:\6171 Power Automedia 401(k) Plan\2023\5500\8955
6180_8955SSA.pdf
6180_8955SSA.pdf
here
File added to PRO for plan 6180.
G:\6180 Pacific Standard Specialties, Inc. 401(k) Profit Sharing Plan\2023\5500\8955
6197_8955SSA.pdf
6197_8955SSA.pdf
here
File added to PRO for plan 6197.
G:\6197 Mills Electric, Co. 401(k) Profit Shari

9830_8955SSA.pdf
here
File added to PRO for plan 9830.
G:\9830 Events BSB 401(k)\2023\5500\8955
9838_8955SSA.pdf
9838_8955SSA.pdf
here
File added to PRO for plan 9838.
G:\9838 QA Consulting Inc\2023\5500\8955
9969_8955SSA.pdf
9969_8955SSA.pdf
here
File added to PRO for plan 9969.
G:\9969 SPEA America\2023\5500\8955
Done!


In [22]:
no_mail_plan_list

['10423',
 '11455',
 '11678',
 '11734',
 '12012',
 '12647',
 '16898',
 '19722',
 '20538',
 '2726',
 '2986',
 '3416',
 '3591',
 '5746',
 '6235',
 '7560',
 '9695']

In [23]:
signatures = pp.get_employees(filters=None)
count_delivered = 0
if df.empty:
    raise SystemExit("Script is shutting down")
for index, row in df.iterrows():
    plan_id = row['planid']
    task_id = row['taskid']
    if not plan_id in no_mail_plan_list:
        starts_with_plan = any(s.startswith(plan_id) for s in final_pdfs)
        if starts_with_plan:
            project_id = row['projid']
            print(plan_id)
            afs_flag = False
            plan_data = pp.get_plan_by_tpaplanid(plan_id)
            count_delivered = count_delivered + 1
            if plan_data['PlanGroupId'] in plan_grp_id_list:
                print('Plan has AFS grouping')
                afs_flag = True
            period_start_date = pp.get_project_by_projectid(project_id)['PeriodStart']
            period_start_date = datetime.datetime.strptime(period_start_date, "%m/%d/%Y %I:%M:%S %p")
            year = str(period_start_date.year)
            try:
                setup_send_mail(plan_id, project_id, signatures, year, afs_flag)
                pp.override_task(task_id)
                df_error_log.loc[df_error_log['TPA Plan ID'].astype(str).str.contains(str(plan_id)), 'Delivered to Client'] = 'Yes'
            except Exception as e:
                print(e)
                continue
        else:
            print('no file for plan: ', plan_id)
    else:
        print('plan in no mail, ', plan_id)
        pp.override_task(task_id)
df_error_log.to_excel(error_path, index = False)    
print('Done!')

APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
12584
Email sent for planid 12584!
Email for 12584 has been copied into the project notes!
11683
Email sent for planid 11683!
Email for 11683 has been copied into the project notes!
10283
Email sent for planid 10283!
Email for 10283 has been copied into the project notes!
11981
The maximum allowed number of requests has been exceeded.
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
Email sent for planid 11981!
Email for 11981 has been copied into the project notes!
plan in no mail,  2726
plan in no mail,  20538
plan in no mail,  6235
10632
Email sent for planid 10632!
Email for 10632 has been copied into the project 

12236
Email sent for planid 12236!
Email for 12236 has been copied into the project notes!
4865
Email sent for planid 4865!
Email for 4865 has been copied into the project notes!
15912
Email sent for planid 15912!
Email for 15912 has been copied into the project notes!
11471
Email sent for planid 11471!
Email for 11471 has been copied into the project notes!
12410
Email sent for planid 12410!
Email for 12410 has been copied into the project notes!
12014
Email sent for planid 12014!
Email for 12014 has been copied into the project notes!
5179
Email sent for planid 5179!
Email for 5179 has been copied into the project notes!
11924
Email sent for planid 11924!
Email for 11924 has been copied into the project notes!
10137
Email sent for planid 10137!
Email for 10137 has been copied into the project notes!
4055
Email sent for planid 4055!
Email for 4055 has been copied into the project notes!
16627
Email sent for planid 16627!
Email for 16627 has been copied into the project notes!
11889
Em

APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
Email sent for planid 3750!
Email for 3750 has been copied into the project notes!
3961
Email sent for planid 3961!
Email for 3961 has been copied into the project notes!
16226
Email sent for planid 16226!
Email for 16226 has been copied into the project notes!
90575
Email sent for planid 90575!
Email for 90575 has been copied into the project notes!
11092
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
Email sent for planid 11092!
Email for 11092 has been copied into the project notes!
5348
Email sent for planid 5348!
Email for 5348 has been copied into the project notes!
17773
Email sent for planid 17773!
Email for

12745
Email sent for planid 12745!
Email for 12745 has been copied into the project notes!
18076
Email sent for planid 18076!
Email for 18076 has been copied into the project notes!
7598
Email sent for planid 7598!
Email for 7598 has been copied into the project notes!
7114
Email sent for planid 7114!
Email for 7114 has been copied into the project notes!
12265
Email sent for planid 12265!
Email for 12265 has been copied into the project notes!
4841
Email sent for planid 4841!
Email for 4841 has been copied into the project notes!
12551
Email sent for planid 12551!
Email for 12551 has been copied into the project notes!
3658
Email sent for planid 3658!
Email for 3658 has been copied into the project notes!
3107
Email sent for planid 3107!
Email for 3107 has been copied into the project notes!
12394
Email sent for planid 12394!
Email for 12394 has been copied into the project notes!
6200
Email sent for planid 6200!
Email for 6200 has been copied into the project notes!
9159
Email sent f

APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
Email sent for planid 4170!
Email for 4170 has been copied into the project notes!
2666
Email sent for planid 2666!
Email for 2666 has been copied into the project notes!
3515
Email sent for planid 3515!
Email for 3515 has been copied into the project notes!
6171
Email sent for planid 6171!
Email for 6171 has been copied into the project notes!
9456
Email sent for planid 9456!
Email for 9456 has been copied into the project notes!
10854
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
Email sent for planid 10854!
Email for 10854 has been copied into the project notes!
91884
Email sent for planid 91884!
Email for 91884 has been copied into the project notes!
6119
Email sent for planid 6119!
Email for 6119 has been copied i

10335
Email sent for planid 10335!
Email for 10335 has been copied into the project notes!
11485
Email sent for planid 11485!
Email for 11485 has been copied into the project notes!
16909
Email sent for planid 16909!
Email for 16909 has been copied into the project notes!
11598
Email sent for planid 11598!
Email for 11598 has been copied into the project notes!
18424
Email sent for planid 18424!
Email for 18424 has been copied into the project notes!
6268
Plan has AFS grouping
Email sent for planid 6268!
Email for 6268 has been copied into the project notes!
11047
Email sent for planid 11047!
Email for 11047 has been copied into the project notes!
Done!


In [24]:
df = pp.get_worktray2('Automation', get_all=True)
filt1 = df['task_name'] == 'Client corrections'
filt2 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df = df[filt1 & filt2]

if df.empty:
    raise SystemExit("Script is shutting down")
for index, row in df.iterrows():
    plan_id = row['planid']
    project_id = row['projid']
    task_id = row['taskid']
    task_id_list = []
    override_flag = False
    print(plan_id)
    tasks = pp.get_task_groups_by_projectid(project_id, expand = 'Tasks')
    for task_group in tasks:
        if task_group['Name'] == 'Automated 8955-SSA Tasks':
            for task in task_group['Tasks']:
                if task['TaskName'] == 'Client corrections' or task['TaskName'] == 'Review Corrected Form 8955-SSA' or task['TaskName'] == 'Delivery of Corrected Form 8955-SSA':
                    task_id_list.append(task['Id'])
                    if task['TaskName'] == 'Client corrections' and task['AssignedToId'] == 834602:
                        override_flag = True
    print(override_flag)
    if override_flag:
        for task_id in task_id_list:
            pp.override_task(task_id)
            print('Task overridden for plan: ',plan_id)
print('Done!')

APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
2018
False
11675
False
10238
True
Task overridden for plan:  10238
Task overridden for plan:  10238
11941
False
10215
False
19225
False
Done!


In [5]:
try:
    source = r'Y:\ASC\Exported Reports\8955 Automation\PDF Downloads'
    for file in os.listdir(source):
        print(file)
        file_path = os.path.join(source, file)
        os.remove(file_path)
except Exception as e:
    print(e)

19199_8955SSA.pdf


In [ ]:
count_str = None
df = pp.get_worktray2('Automation', get_all=True)
filt1 = df['task_name'] == 'Delivery of Form 8955-SSA'
filt2 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df1 = df[filt1 & filt2]
count_delivery = len(df1)
filt3 = df['task_name'] == 'Completion - Form 8955-SSA'
filt4 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df2 = df[filt3 & filt4]
count_completion = len(df2)
filt5 = df['task_name'] == 'Client corrections'
filt6 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df3 = df[filt5 & filt6]
count_correction = len(df3)
try:
    count_per_column = df_error_log.count()
    count_dict = count_per_column.to_dict()
    count_dict = {'Number of Plans' if key == 'TPA Plan ID' else key: value for key, value in count_dict.items()}
    count_dict['Form Downloaded'] = count_delivered
    count_dict['Delivered to Client'] = count_delivered
    count_dict['Number of plans in Completion - Form 8955-SSA task'] = count_completion
    count_dict['Number of plans in Delivery of Form 8955-SSA task'] = count_delivery
    count_dict['Number of plans in Client corrections task'] = count_correction
    count_str = "\n".join([f"{column}: {count}" for column, count in count_dict.items()])
    print(count_str)
except Exception as e:
    print(e)

In [ ]:
mail_daily_report(error_path, count_str)

In [25]:
end = datetime.datetime.now()
diff = end - start 
minutes = diff.total_seconds() / 60
print('minutes: ',minutes)

minutes:  150.59845298333332
